In [77]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [78]:
# from sklearn.model_selection import train_test_split    
df1 = pd.read_csv('main_dataset.csv')
df2 = pd.read_csv('drug_dict_onehot.csv')
df1.head()


,Unnamed: 0,inpatient.number,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,...,carboxyhemoglobin,body.temperature.blood.gas,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0,857781,1.0,0.242857,0.439394,0.527778,0.404762,0.438356,0.422794,0.046006,...,0.075472,NaN,0.96,0.310638,0.957219,0.527778,0.5,0.535714,1.0,0.666667
1,1,743087,0.0,0.257143,0.479798,0.500000,0.595238,0.479452,0.533088,0.047504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
2,2,866418,1.0,0.214286,0.494949,0.500000,0.404762,0.458904,0.433824,0.059942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.533333
3,3,775928,1.0,0.142857,0.368687,0.527778,0.436508,0.506849,0.474265,0.055660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,4,810128,0.0,0.000000,0.444444,0.527778,0.531746,0.424658,0.474265,0.078285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667


In [79]:
df1 = df1.sort_values(by=['inpatient.number'], ascending=True)
df1.reset_index(drop = True, inplace = True)
df1.head()

,Unnamed: 0,inpatient.number,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,...,carboxyhemoglobin,body.temperature.blood.gas,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,1057,722128,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
1,797,723327,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,...,0.132075,NaN,0.880000,0.187234,0.878342,0.394444,0.44,0.226190,1.0,0.533333
2,296,723617,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,...,0.000000,NaN,0.973333,0.365957,0.981283,0.286111,0.44,0.410714,1.0,0.800000
3,1372,724385,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,953,725509,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.266667


In [80]:
df2 = df2.sort_values(by=['inpatient.number'], ascending=True)
df2.reset_index(drop = True, inplace = True)
df2.head()

,Unnamed: 0,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [81]:
pd.concat([df1['inpatient.number'],df2['inpatient.number']]).drop_duplicates(keep=False)

Series([], Name: inpatient.number, dtype: int64)

In [82]:
df1['inpatient.number'].equals(df2['inpatient.number'])

True

In [83]:
df1.drop(['inpatient.number','Unnamed: 0'],axis= 1,inplace=True)
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,carboxyhemoglobin,body.temperature.blood.gas,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.132075,NaN,0.880000,0.187234,0.878342,0.394444,0.44,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.000000,NaN,0.973333,0.365957,0.981283,0.286111,0.44,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.266667


In [84]:
df2.drop(['inpatient.number','Unnamed: 0'],axis= 1,inplace=True)
df2.head()
len(df2.columns)

18

In [85]:
X = df1
y = df2


In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
len(X), len(y_train), len(y_test)

(1996, 1596, 400)

In [106]:
model = Sequential()
model.add(Dense(30, input_dim=146, activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')


In [112]:
model.fit(X_train, y_train, epochs = 1)


50/50 [==============================] - 0s 997us/step - loss: nan
